In [1]:
import os 
import yaml
from datetime import datetime
import mujoco.egl
import sys
sys.path.append("..")
from algorithms.utils.wrapper_gym import get_env
from algorithms.offline.any_percent_bc import get_actor_from_checkpoint, eval_actor


In [2]:
os.environ["XLA_PYTHON_CLIENT_PREALLOCATE"] = "false"
os.environ["MUJOCO_GL"] = "egl"

In [3]:
gl_context = mujoco.egl.GLContext(1024, 1024)
gl_context.make_current()

In [4]:
import subprocess
import zipfile

def download_from_gdrive(gdrive_url, output_path):
    try:
        import gdown
    except ImportError:
        subprocess.check_call([sys.executable, "-m", "pip", "install", "gdown"])
        import gdown
    gdown.download(gdrive_url, output_path, quiet=False)

GDRIVE_URL = "https://drive.google.com/uc?id=1I6X9Zn0-ha3kINq4i_xn9112jDFPlnlK"
OUTPUT_PATH = "/CORL/sim2real/BC-Go2JoystickFlatTerrain.zip"
if not os.path.exists(OUTPUT_PATH):
    download_from_gdrive(GDRIVE_URL, OUTPUT_PATH)

with zipfile.ZipFile(OUTPUT_PATH, 'r') as zip_ref:
    zip_ref.extractall("/CORL/sim2real/")

In [5]:
CHECKPOINT_PATH = "/CORL/sim2real/BC-Go2JoystickFlatTerrain-d7c8b162"
ENV_NAME = "Go2JoystickFlatTerrain"

N_EPISODES = 20
RENDER = True
SEED = 0

In [6]:
with open(os.path.join(CHECKPOINT_PATH, "config.yaml")) as f:
    config = yaml.safe_load(f)

In [7]:
render_trajectory = []
def render_callback(_, state):
    render_trajectory.append(state)


# Command type: foward, fowardfixed, direction
# Foward: move forward with random velocity
# Fowardfixed: move forward with fixed velocity
# Direction: move in a specific direction
env_wrapped = get_env(ENV_NAME, "cuda", render_callback, command_type="fowardfixed")

state_dim = env_wrapped.observation_space.shape[0] 
action_dim = env_wrapped.action_space.shape[0]
max_action = 1.0

print(state_dim, action_dim, max_action)

obs_shape: {'privileged_state': (123,), 'state': (48,)}
Asymmetric observation space
JITing reset and step
Done JITing reset and step
48 12 1.0


In [8]:
actor = get_actor_from_checkpoint(checkpoint_path=CHECKPOINT_PATH, state_dim=state_dim, action_dim=action_dim, max_action=max_action)

[get_actor_from_checkpoint] Loaded actor device: cuda:0


In [9]:
episode_rewards = eval_actor(actor=actor, env=env_wrapped, device=config["device"], n_episodes=N_EPISODES, seed=SEED, render=RENDER)
print("Mean: ", episode_rewards.mean())
print("Std: ", episode_rewards.std())

Mean:  23.555313940507222
Std:  8.132073290181596


In [10]:
timestamp = datetime.now().strftime("%Y%m%d-%H%M%S")
env_wrapped.save_video(render_trajectory, save_path=f"{CHECKPOINT_PATH}/{ENV_NAME}-{timestamp}.mp4")

/usr/local/lib/python3.10/dist-packages/glfw/__init__.py:917: GLFWError: (65550) b'X11: The DISPLAY environment variable is missing'
  warnings.warn(message, GLFWError)
/usr/local/lib/python3.10/dist-packages/glfw/__init__.py:917: GLFWError: (65537) b'The GLFW library is not initialized'
  warnings.warn(message, GLFWError)
100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 8644/8644 [15:10<00:00,  9.49it/s]
